In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import tri
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

In [2]:
# Step 1: Define problem parameters
L = 50  # Length of domain in x and y directions
Nx, Ny = 50, 50  # Number of grid points in x and y
alpha = 2  # Thermal diffusivity
T = 1.0  # Total time
dt = 0.125  # Time step size
nt = 100  # Number of time steps

In [3]:
# Step 2: Generate grid (structured triangular mesh)
x = np.linspace(0, L, Nx)
y = np.linspace(0, L, Ny)
X, Y = np.meshgrid(x, y)
triangulation = tri.Triangulation(X.flatten(), Y.flatten())

In [4]:
# Step 3: Initial condition
u = np.zeros((L * L,))
u[:L] = 100

In [5]:
def assemble_matrices(triangulation, alpha):
    npoints = len(triangulation.x)  # Total number of points in the mesh
    K = np.zeros((npoints, npoints))  # Global stiffness matrix
    M = np.zeros((npoints, npoints))  # Global mass matrix
    
    for element in triangulation.triangles:
        # Get the coordinates of the three vertices of the triangle
        vertices = element[:]
        x_coords = triangulation.x[vertices]
        
        y_coords = triangulation.y[vertices]
        # Compute area of the triangle (used in both stiffness and mass matrices)
        area = 0.5 * abs(
            x_coords[0]*(y_coords[1]-y_coords[2]) +
            x_coords[1]*(y_coords[2]-y_coords[0]) +
            x_coords[2]*(y_coords[0]-y_coords[1])
        )
        
        # Local stiffness matrix (based on gradients of linear basis functions)
        Ke = (alpha / (4 * area)) * np.array([[2, -1, -1],
                                              [-1, 2, -1],
                                              [-1, -1, 2]])
        
        # Local mass matrix (based on linear basis functions)
        Me = (area / 12) * np.array([[2, 1, 1],
                                     [1, 2, 1],
                                     [1, 1, 2]])
        
        # Add local contributions to the global matrices
        for i in range(3):
            for j in range(3):
                K[vertices[i], vertices[j]] += Ke[i, j]
                M[vertices[i], vertices[j]] += Me[i, j]
    
    return K, M

In [6]:
K, M = assemble_matrices(triangulation, alpha)

In [8]:
# Step 5: Time-stepping loop (Forward Euler)
M_inv = np.linalg.inv(M)
for n in range(nt):
    u_new = u - dt * np.dot(M_inv, np.dot(K, u))
    u = u_new
    print(u[0])

-17.71059561021883
-15.410436014695506
1804.7876906783856
-16865.280911174803
128939.80750762363
-904599.3998333768
6089718.591110184
-40054648.581746325
259733915.7836581
-1668307198.4802694
10641713170.234516
-67504543198.54646
426117664029.41
-2677261974758.097
16740303755243.91
-104129784402893.19
643931901544979.2
-3954995495171737.0
2.4094826648697964e+16
-1.4534228996274845e+17
8.658914962742825e+17
-5.076720641450403e+18
2.9136216193048773e+19
-1.6231183772482857e+20
8.65066235270815e+20
-4.288748731646121e+21
1.8497377157589299e+22
-5.423557132411943e+22
-1.1222161621848257e+23
4.004558005471708e+24
-4.912377731040112e+25
4.8223741162800336e+26
-4.284574975883282e+27
3.597000985495959e+28
-2.9115839614396888e+29
2.297706332169225e+30
-1.7797961068444594e+31
1.35917225774561e+32
-1.0264294075977204e+33
7.682151052712545e+33
-5.707412633487895e+34
4.214381901125718e+35
-3.095874269734424e+36
2.264215127849872e+37
-1.6496915039376473e+38
1.1979957142717827e+39
-8.674695550995073e

In [9]:
u.shape

(2500,)

In [10]:
u

array([ 9.49598026e+84,  6.05189848e+84, -1.52565326e+85, ...,
       -4.13766219e+76,  8.85035545e+76, -1.12156528e+76])